<h2 align='center'>Codebasics ML Course: ML Flow Tutorial</h2>

In [6]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [7]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100], dtype=int64))

In [8]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

#### Handle class imbalance

In [9]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)
np.unique(y_train_res, return_counts=True)

(array([0, 1]), array([619, 619], dtype=int64))

### Track Experiments

In [10]:
models = [
    (
        "Logistic Regression", 
        {"C": 1, "solver": 'liblinear'},
        LogisticRegression(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest", 
        {"n_estimators": 30, "max_depth": 3},
        RandomForestClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [11]:
reports = []

for model_name, params, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    
    model.set_params(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [12]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost

In [13]:
pip install dagshub

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/236.6 kB ? eta -:--:--
   ---------------------------------------- 236.6/236.6 kB 7.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/71.5 kB ? eta -:--:--
   ---------------------------------------- 71.5/71.5 kB 1.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/238.4 kB ? eta -:--:--
   ---------------------------------------  235.5/238.4 kB 7.3 MB/s eta 0:00:01
   ---------------------------------------- 238.4/238.4 kB 3.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/139.2 kB ? eta -:--:--
   ---------------------------------------- 139.2/139.2 kB 4.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.5 MB ? eta -:--:--
   - -------------------------------------- 0.6/12.5 MB 17.8 MB/s eta 0:00:01
   --- ------------------------------------ 1.0/12.5 MB 11.0 MB/s eta

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.7.0 requires botocore<1.31.65,>=1.31.16, but you have botocore 1.34.158 which is incompatible.


In [15]:
# dagshub setup
import dagshub
dagshub.init(repo_owner='saurabhsawant28', repo_name='mlflow_dagshub_demo', mlflow=True)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=b9695da6-c0b6-4119-8bbb-5b6f1e923c05&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=1617f8e7413c1707bf33d03e54237c08daf0d0c15f7b0c1656e99da16602214d




Output()

Accessing as saurabhsawant28

Initialized MLflow to track repo "saurabhsawant28/mlflow_dagshub_demo"

Repository saurabhsawant28/mlflow_dagshub_demo initialized!

In [16]:
# Initialize MLflow
mlflow.set_experiment("Anomaly Detection")
mlflow.set_tracking_uri("https://dagshub.com/saurabhsawant28/mlflow_dagshub_demo.mlflow")

for i, element in enumerate(models):
    model_name = element[0]
    params = element[1]
    model = element[2]
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):        
        mlflow.log_params(params)
        mlflow.log_metrics({
            'accuracy': report['accuracy'],
            'recall_class_1': report['1']['recall'],
            'recall_class_0': report['0']['recall'],
            'f1_score_macro': report['macro avg']['f1-score']
        })  
        
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")  

2024/08/12 14:34:54 INFO mlflow.tracking.fluent: Experiment with name 'Anomaly Detection' does not exist. Creating a new experiment.
2024/08/12 14:35:21 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/08/12 14:35:21 INFO mlflow.tracking._tracking_service.client: 🏃 View run Logistic Regression at: https://dagshub.com/saurabhsawant28/mlflow_dagshub_demo.mlflow/#/experiments/0/runs/f6810279c2fe4595b9f354fd9a622976.
2024/08/12 14:35:21 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/saurabhsawant28/mlflow_dagshub_demo.mlflow/#/experiments/0.
2024/08/12 14:35:39 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/08/12 14:35:39 INFO mlflow.tracking._tracking_service.client: 🏃 View run Random Forest at: https://dagshub.com/saurabhsawant28/mlflow

### Register the Model

In [ ]:
model_name = 'XGB-Smote'
run_id=input('Please type RunID')
model_uri = f'runs:/{run_id}/model_name'

with mlflow.start_run(run_id=run_id):
    mlflow.register_model(model_uri=model_uri, name=model_name)

### Load the Model

In [41]:
model_version = 1
model_uri = f"models:/{model_name}/{model_version}"

loaded_model = mlflow.xgboost.load_model(model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])

### Transition the Model to Production

In [42]:
current_model_uri = f"models:/{model_name}@challenger"
production_model_name = "anomaly-detection-prod"

client = mlflow.MlflowClient()
client.copy_model_version(src_model_uri=current_model_uri, dst_name=production_model_name)

<ModelVersion: aliases=[], creation_timestamp=1722288798033, current_stage='None', description='', last_updated_timestamp=1722288798033, name='anomaly-detection-prod', run_id='3e6b0bdb88ac4df9add7a826594d8d33', run_link='', source='models:/XGB-Smote/1', status='READY', status_message='', tags={}, user_id='', version='1'>

In [43]:
model_version = 1
prod_model_uri = f"models:/{production_model_name}@champion"

loaded_model = mlflow.xgboost.load_model(prod_model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])

Please refer to following to learn more about model registry

https://mlflow.org/docs/latest/model-registry.html#model-registry-workflows to learn 